RAG chatbot 
Version 1.0 

Import Libraries

In [20]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_classic.retrievers import MultiQueryRetriever
import chromadb

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Suppress ChromaDB telemetry warnings as they are useless as ChromaDB trying to send usage data but failing because it can't find its own version information
import logging
logging.getLogger("chromadb").setLevel(logging.ERROR)

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

LOAD PDF

In [21]:
# Load PDF
doc_path = "TaskFlow.pdf"
if doc_path:
    loader = UnstructuredPDFLoader(file_path=doc_path, languages=["deu"])
    data = loader.load()
    print(f"PDF loaded successfully: {doc_path}")
else:
    print("Upload documents")

#TODO: language detection!!!!


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


PDF loaded successfully: TaskFlow.pdf


SPLIT TEXT INTO CHUNKS

In [22]:
#Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Number of text chunks: {len(chunks)}")

Number of text chunks: 6


CREATE VECTOR DATABASE

In [23]:
# Create vector database
client = chromadb.PersistentClient(path="./chroma_data")
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="rag_chatbot_collection",
    client=client
)
print("Vector database created successfully.")


Vector database created successfully.


Set up LLM and Retrieval

In [24]:
#Set up LLM and retrieval
local_model = "llama3.2"
llm = ChatOllama(model=local_model, temperature=0.1)

# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are a query rephrasing assistant for a vector database search.
    Your task is to generate **3** alternative, semantically diverse versions of the user's question. These variations should explore different terminology, structures, or angles of the original query to maximize relevant document retrieval.
    Provide only the rephrased questions, with each one separated by a newline. Do not include any introductory phrases, explanations, or the original question itself.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

Create chain

In [25]:
# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


# Create chain (pipeline)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Chat with PDF

In [26]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [27]:
# Example 1
chat_with_pdf("What is the main idea of this document?")

The main idea of this document appears to be a specification or requirements document for a task management system, outlining the functional and non-functional requirements for its development and implementation. The document covers various aspects such as user interface, data storage, API functionality, performance, and usability, among others.

In [28]:
# Example 2
chat_with_pdf("What are the two options for technical implementation of this system?")

The two options for technical implementation of this system are:

1. Implementierung als Webanwendung (Spring o.ä.)
2. Implementierung als Desktop Anwendung (Swing)

In [29]:
# Example 4
chat_with_pdf("What should I do? Can you give me some steps?")

Based on the provided context, it seems that you are working with a TaskFlow system, which is a task management system for employees in companies. The document provides requirements and specifications for the system.

To answer your question, "What should I do?", I'll provide some general steps based on the context:

1. **Review the requirements**: Carefully read through the document's page content to understand the functional, performance, security, and maintainability requirements of the TaskFlow system.
2. **Identify areas for improvement**: Based on your understanding of the requirements, identify areas where you can improve or optimize the system, such as performance bottlenecks or security vulnerabilities.
3. **Develop a plan**: Create a plan to address the identified areas, including any necessary changes to the code, data storage, or API implementation.
4. **Implement changes**: Start implementing the planned changes, following the modular architecture and testing guidelines outlined in the document.
5. **Test and validate**: Thoroughly test the updated system to ensure it meets the requirements and performs well under various scenarios.
6. **Monitor and maintain**: Continuously monitor the system's performance and security, making adjustments as needed to maintain the required availability and reliability.

Please note that these steps are general suggestions based on the provided context and may not be specific to your exact situation. If you have more information or clarification about your project, I'd be happy to provide more tailored advice.